<a href="https://colab.research.google.com/github/Banking-Analytics-Lab/DLinBankingBook/blob/main/Labs/TextBook_Lab_Chap5_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Default correlation in mortgage lending

This notebook demonstrates the application of Graph Neural Networks (GNNs) for predicting mortgage defaults using loan-level data from Freddie Mac.

The notebook is in two parts which you can run independently. The first part focuses on processing the data to create the networks whereas the second part focuses on building and training the GNN models using PyTorch Geometric.

We start by loading the necessary libraries and the data.

In [ ]:
!pip install torch_geometric

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
from tqdm import tqdm
import PIL
import os
import random
import pickle

# Plots
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve, auc,accuracy_score

import torch
import torch.nn as nn

import torch_geometric
from torch_geometric.utils import from_networkx, degree
from torch_geometric.data import Data
from torch_geometric.transforms import RandomNodeSplit
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv, SAGEConv

In [ ]:
!gdown '1YadoR0hR_uZJe5XyGhh0PUwjqfkeJu7k'

## Part 1: Data processing and network building
A dataset similar to the one for the time series lab has been prepared. We read it in and look at the first few lines.

In [ ]:
data=pd.read_csv("graph_df.csv")
data.head()

We will use the AREA and and PROVIDER variables as connector variables to build our network. They are both categorical variables. But we need to do some preprocessing before we continue

First, we will remove all the categories which only have a single borrower as the would end up a singletons in our network.

In [ ]:
area_counts = data['AREA'].value_counts()
data = data[data['AREA'].isin(area_counts[area_counts > 1].index)].reset_index(drop=True)

For the PROVIDER variable we remove the value Other sellers as it may introduce unnecessary noise.

In [ ]:
data = data[data['PROVIDER'] != 'Other sellers'].reset_index(drop=True)

Now, we create the edge lists for our networks. To do this, we loop over all the different values in the connector variables and identify which borrowers share values. We do this separately for the AREA variable and the PROVIDER variable.

Note that there are two variants in the next two cells. One to create the full edgelist which is used in the analyses, and one to create a smaller edge list (with only 1000 borrowers) which is used to plot the network. You can comment out the lines you do not need.

Running the cells for the full edge list can take a long time and therefore we recommend saving the result at the end. This line is commented out below.

In [ ]:
#area edge list
el = []
#for i in tqdm(range(len(data))): #For full edgelist
#   for j in range(len(data)): #For full edgelist
for i in tqdm(range(1000)): #For smaller edgelist
    for j in range(1000): #For smaller edgelist
        if data.loc[i, 'AREA'] == data.loc[j, 'AREA']:
            el.append((data.loc[i,'LOAN_NUMBER'], data.loc[j,'LOAN_NUMBER']))
el_df = pd.DataFrame(el)
index = el_df[el_df.loc[:, 0] == el_df.loc[:, 1]].index
el_df.drop(index, inplace = True)
el_df.reset_index(drop = True, inplace = True)
el_df.rename(columns = {0:'source', 1:'target'}, inplace = True)
#el_df.to_csv('areaEdgeList.csv', index = False)
area=el_df

In [ ]:
el = []
#for i in tqdm(range(len(data))):#For full edgelist
    #for j in range(len(data)):#For full edgelist
for i in tqdm(range(1000)):#For smaller edgelist
   for j in range(1000):#For smaller edgelist
        if data.loc[i, 'PROVIDER'] == data.loc[j, 'PROVIDER']:
            el.append((data.loc[i,'LOAN_NUMBER'], data.loc[j,'LOAN_NUMBER']))
el_df = pd.DataFrame(el)
index = el_df[el_df.loc[:, 0] == el_df.loc[:, 1]].index
el_df.drop(index, inplace = True)
el_df.reset_index(drop = True, inplace = True)
el_df.rename(columns = {0:'source', 1:'target'}, inplace = True)
#el_df.to_csv('providerEdgeList.csv', index = False)
comp=el_df

Next, we combine the two edgelists into one.

In [ ]:
edge_df = pd.concat([area, comp], axis = 0)
edge_df.shape

We create a network object from our edgelist and add the features as node attributes and then relabel the nodes.

In [ ]:
G_areacomp = nx.from_pandas_edgelist(edge_df, source = "source", target = "target")
G_areacomp.number_of_nodes(), G_areacomp.number_of_edges()

In [ ]:
for i in tqdm(G_areacomp.nodes()):
    id_loan = data[data['LOAN_NUMBER'] == i]['LOAN_NUMBER'].tolist()[0]
    for f in data.columns[0:23]:


        G_areacomp.nodes[i][f] = data[f][data['LOAN_NUMBER'] == i].tolist()[0]

In [ ]:
mapping = {old_label: new_label for new_label, old_label in enumerate(G_areacomp.nodes())}
H_relabel = nx.relabel_nodes(G_areacomp, mapping)

We can also plot our network (this is only recommneded for the smaller network). We can see that defaulted borrowers (the dark gray nodes) are somewhat clustered together. This indicates that  default is correlated and motivates our approach of using the network information to predict default.

In [ ]:
node_colors = [
    'black' if H_relabel.nodes[n].get('target', 0) == 1 else 'grey'
    for n in H_relabel.nodes()
]

plt.figure(figsize=(12, 8))
pos = nx.spring_layout(H_relabel, seed=42)
nx.draw(
    H_relabel,
    pos,
    node_size=40,
    node_color=node_colors,
    edge_color='lightgray',
    width=0.5,
    with_labels=False
)
plt.show()

To avoid long running times, we are here loading the edgelists of the full dataset. You can therefore skip the long-running for loops above.
We then continue to create the network and add the features as node attributes as above.

In [ ]:
!gdown '17wG2Bo0ENVU74Rz5UI2y2jbZc4rfNSHU'

In [ ]:
!gdown '1Nk_eMZLrkfCmdwHv5OCax2gh52mUZVxw'

In [ ]:
area    = pd.read_csv('areaEdgeList.csv')
comp    = pd.read_csv('providerEdgeList.csv')

In [ ]:
edge_df = pd.concat([area, comp], axis = 0)
edge_df.shape

In [ ]:
G_areacomp = nx.from_pandas_edgelist(edge_df, source = "source", target = "target")
G_areacomp.number_of_nodes(), G_areacomp.number_of_edges()

In [ ]:
for i in tqdm(G_areacomp.nodes()):
    id_loan = data[data['LOAN_NUMBER'] == i]['LOAN_NUMBER'].tolist()[0]
    for f in data.columns[0:23]:
        G_areacomp.nodes[i][f] = data[f][data['LOAN_NUMBER'] == i].tolist()[0]

In [ ]:
mapping = {old_label: new_label for new_label, old_label in enumerate(G_areacomp.nodes())}
H_relabel = nx.relabel_nodes(G_areacomp, mapping)

Finally, we save our network object to not have to run the above code again.

In [ ]:
with open('network.gpickle', 'wb') as f:
    pickle.dump(H_relabel, f, pickle.HIGHEST_PROTOCOL)

## Part 2: GNN models
To start our model building we load the network object we created in the previous part.

Note: you can start running the notebook from here.

In [ ]:
!gdown 'https://drive.google.com/uc?id=1-AOl7sCEusPqmctnStdPg8Frc9XEOcKf'

In [ ]:
with open('network.gpickle', 'rb') as f:
    G = pickle.load(f)

### Data preparation

Next, we create a PyG Data object, starting with adding the following elements
- edgelist: the argument `edge_index`
- target variable: the argument `y`
- node attributes: the argument `x`

In [ ]:
edge_list = nx.to_pandas_edgelist(G)
edge_i = torch.tensor(edge_list.values, dtype=torch.long).t().contiguous()
myData = Data(edge_index=edge_i)

In [ ]:
target_attribute = list(nx.get_node_attributes(G, 'target').values())
myData.y = torch.tensor(target_attribute, dtype=torch.long)

In [ ]:
node_data = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')
tmp=node_data.iloc[:,0:21]
myData.x=torch.tensor(tmp.values)

To facilitate the GNNs learning we add the nodes' degree as a node attribute.



This step can be skipped.

In [ ]:
edge_index = myData.edge_index
num_nodes = myData.num_nodes
deg = degree(edge_index[0], num_nodes=num_nodes)
deg = deg.view(-1, 1).float()
myData.x = torch.cat([myData.x, deg], dim=1)

In [ ]:
myData.x.shape  # Check the shape of the updated node features

Next we create the masks for training, validation and test.

In [ ]:
node_transform = RandomNodeSplit(split='train_rest',num_val=10000, num_test=10000,num_train_per_class=900)
node_splits = node_transform(myData)
node_splits.x = node_splits.x.float()
node_splits.y = node_splits.y.float()

As a sanity check, we do some inspection of our network.

In [ ]:
print()
print(f'Dataset: {node_splits}:')
print('======================')
print(f'Number of features: {node_splits.num_features}')
#print(f'Number of classes: {myData.num_classes}')
print(f'Number of defaults: {(node_splits.y.sum())}')
#===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {node_splits.num_nodes}')
print(f'Number of edges: {node_splits.num_edges}')
print(f'Average node degree: {node_splits.num_edges / node_splits.num_nodes:.2f}')
print(f'Number of training nodes: {node_splits.train_mask.sum()}')
print(f'Number of training nodes: {node_splits.val_mask.sum()}')
print(f'Number of training nodes: {node_splits.test_mask.sum()}')
print(f'Training default rate: {node_splits.y[node_splits.train_mask].sum() / node_splits.train_mask.sum():.2f}')
print(f'Validation default rate: {node_splits.y[node_splits.val_mask].sum() / node_splits.val_mask.sum():.2f}')
print(f'Test default rate: {node_splits.y[node_splits.test_mask].sum() / node_splits.test_mask.sum():.2f}')
print(f'Has isolated nodes: {node_splits.has_isolated_nodes()}')
print(f'Has self-loops: {node_splits.has_self_loops()}')
print(f'Is undirected: {node_splits.is_undirected()}')


Finally, we scale the node attributes which is usually a good idea before training any neural network.

In [ ]:
scaler = StandardScaler()
node_splits.x = torch.tensor(scaler.fit_transform(node_splits.x.numpy()), dtype=torch.float)

### Model definition

We are now ready to define our GNN models. We use three different GNN models
1. GraphSAGE
2. Graph Convolutional Networks (GCN)
3. Graph Attention Networks (GAT)

We create a separate class for each model.
The classes contain a constructor and a forward method an give a single output which is the predicted probability of default.
All the models have the same decoder.

The models as defined here have gone through parameter tuning, where amongst others different number of GNN and linear layers, different dimensions of hidden and dense layers and differnt levels of dropout where tried. You can add more layers or other functionality to the architecture to see how it imapcts the performance.

In [ ]:
# 1. GraphSAGE model
class BinaryGraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels,dense_channels):
        super(BinaryGraphSAGE, self).__init__()
        self.sage1 = SAGEConv(in_channels, hidden_channels)
        self.sage2 = SAGEConv(hidden_channels, hidden_channels)


        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(hidden_channels, dense_channels),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.5),
            torch.nn.Linear(dense_channels, 1)
        )
    def forward(self, x, edge_index):
        x = self.sage1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.sage2(x, edge_index)
        x=F.relu(x)
        out = self.decoder(x)
        return out.view(-1)

# 2. GCN model
class BinaryGCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels,hidden_dense):
        super(BinaryGCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)

        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(hidden_channels, hidden_dense),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.5),
            torch.nn.Linear(hidden_dense, 1)
        )

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        out = self.decoder(x)
        return out.view(-1)

# 3. GAT model
class BinaryGAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, dense_channels,heads):
        super(BinaryGAT, self).__init__()
        self.gat1 = GATConv(in_channels, hidden_channels, heads=heads,concat=False, dropout=0.5)
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(hidden_channels, dense_channels),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.5),
            torch.nn.Linear(dense_channels, 1)
        )

    def forward(self, x, edge_index):
        x = self.gat1(x, edge_index)
        x = F.elu(x)
        out = self.decoder(x)
        return out.view(-1)


### Training procedure
Next we set up our procedure to train the models.

- In each epoch, the data is passed through the model, the loss is calculated and backpropagated through the model.
- We evaluate the model using accuracy, auc and the confusion matrix.
- The training function includes an early stopping functionality with patience
- At the end, the best model is loaded and evaluated using the test set.

In the following cells we define our loss function and set up our learning.

In [ ]:
def train_one_epoch(model, data, optimizer, loss_fn):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = loss_fn(out[data.train_mask], data.y[data.train_mask].float())
    loss.backward()
    optimizer.step()
    return loss.item()


@torch.no_grad()
def evaluate(model, data, mask, loss_fn):
    model.eval()
    out = model(data.x, data.edge_index)
    probs = torch.sigmoid(out)
    preds = (probs > 0.5).float()
    y_true = data.y[mask].cpu()
    y_probs = probs[mask].cpu()
    y_preds = preds[mask].cpu()
    acc = accuracy_score(y_true, y_preds)

    try:
        auc = roc_auc_score(y_true, y_probs)
        fpr, tpr, thresholds = roc_curve(y_true, y_probs)

    except ValueError:
        auc = float('nan')

    conf_mat = confusion_matrix(y_true, y_preds)

    val_loss = loss_fn(out[mask], data.y[mask].float()).item()
    return acc, auc, fpr,tpr, conf_mat, val_loss


def train_model(model, data, loss_fn, optimizer, patience=20,
                max_epochs=1000, save_path="best_model.pt"):
    best_val_loss = 1e10
    patience_counter = 0
    best_model_state = None
    losses = []
    val_losses = []

    for epoch in range(max_epochs):
        loss = train_one_epoch(model, data, optimizer, loss_fn)
        losses.append(loss)
        _, val_auc,val_fpr,val_tpr, _, val_loss = evaluate(model, data, data.val_mask, loss_fn)
        val_losses.append(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict()
            patience_counter = 0
            torch.save(model.state_dict(), save_path)  # Save best model to disk

        else:
            patience_counter += 1
            if patience_counter >= patience:
                break

    model.load_state_dict(torch.load(save_path))  # Load best model from disk
    test_acc, test_auc, test_fpr,test_tpr,conf_mat, _ = evaluate(model, data, data.test_mask, loss_fn)
    return losses, val_losses, test_acc, test_auc, test_fpr,test_tpr,conf_mat

We set things up with the appropriate device and send the data there, as is standard.


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = node_splits.to(device)

Our data is heavily imbalanced, with only 2% default rate. Therefore we use the binary cross entropy loss function with sigmoid activation where we also use the `pos_weight` argument, which we set equal to the imbalance ratio in our training set.

In [ ]:
y_train = data.y[data.train_mask]
num_pos = (y_train == 1).sum().item()
num_neg = (y_train == 0).sum().item()
pos_weight = torch.tensor([num_neg / num_pos], device=device)
loss_fn = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)

#### Training GraphSAGE
We train the GraphSAGE model and print the performance. The parameters are as follows:
- `in_channels`=22: the number of node attributes.
- `hidden_channels`=4: The dimension of the GNN embeddings. This model has 2 GNN layers.
- `dense_channels`=4: The dimension of the dense embeddigns in the decoder.

The loss curves show that the model learns quite well, and we get a good performance, with AUC of around 0.80 on the test set.

In [ ]:
model = BinaryGraphSAGE(22, 4,4).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)
losses, val_losses, acc, auc,fpr,tpr, cm = train_model(model, data, loss_fn, optimizer,
                                                   patience=100, max_epochs=1000,
                                                   save_path=f"best_GraphSAGE.pt")
print(f"Test Accuracy: {acc:.3f}")
print(f"Test AUC:      {auc:.3f}")

fig, axes = plt.subplots(1, 3, figsize=(24, 6))
# Plot Loss Curves
axes[0].plot(losses, label="Train")
axes[0].plot(val_losses, label="Val")
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Loss")
axes[0].set_title("GraphSAGE Loss Curves")
axes[0].legend()

# Plot ROC Curve
axes[1].plot(fpr, tpr, lw=2, label=f'AUC = {auc:.3f}')
axes[1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
axes[1].set_xlim([0.0, 1.0])
axes[1].set_ylim([0.0, 1.05])
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('GraphSAGE Receiver Operating Characteristic (ROC) Curve')
axes[1].legend(loc="lower right")
axes[1].grid(True)

# Plot Confusion Matrix
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', ax=axes[2])
axes[2].set_xlabel('Predicted')
axes[2].set_ylabel('True')
axes[2].set_title('GraphSAGE Confusion Matrix')

plt.tight_layout()
plt.show()

#### Training GCN
We train the GCN model and print the performance. The parameters are as follows:
- `in_channels`=22: the number of node attributes
- `hidden_channels`=16: The dimension of the GNN embeddings
- `dense_channels`=4: The dimension of the dense embeddigns in the decoder

The learning parameters here are very different from those of the GraphSAGE and GAT models, as this model struggles to learn the structure in the data. This is evident from the loss curves and the low AUC value.

In [ ]:
model = BinaryGCN(22, 16, 4).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-4)
losses, val_losses, acc, auc,fpr,tpr, cm = train_model(model, data, loss_fn, optimizer,
                                                   patience=100, max_epochs=100000,
                                                   save_path=f"best_GCN.pt")

print(f"Test Accuracy: {acc:.3f}")
print(f"Test AUC:      {auc:.3f}")

fig, axes = plt.subplots(1, 3, figsize=(24, 6))
# Plot Loss Curves
axes[0].plot(losses, label="Train")
axes[0].plot(val_losses, label="Val")
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Loss")
axes[0].set_title("GCN Loss Curves")
axes[0].legend()

# Plot ROC Curves
axes[1].plot(fpr, tpr, lw=2, label=f'AUC = {auc:.3f}')
axes[1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
axes[1].set_xlim([0.0, 1.0])
axes[1].set_ylim([0.0, 1.05])
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('GCN Receiver Operating Characteristic (ROC) Curve')
axes[1].legend(loc="lower right")
axes[1].grid(True)

# Plot Confusion Matrix
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', ax=axes[2])
axes[2].set_xlabel('Predicted')
axes[2].set_ylabel('True')
axes[2].set_title('GCN Confusion Matrix')

plt.tight_layout()
plt.show()

#### Training GAT
We train the GAT model and print the performance. The parameters are as follows:
- `in_channels`=22: the number of node attributes
- `hidden_channels`=8: The dimension of the GNN embeddings
- `dense_channels`=4: The dimension of the dense embeddigns in the decoder
- `heads`=2: The number of attention heads

The loss curves show that the model learns quite well, and we get a good performance, with AUC slightly below that of GraphSAGE.

In [ ]:
model = BinaryGAT(22, 8,4, 2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)
losses, val_losses, acc, auc,fpr,tpr, cm = train_model(model, data, loss_fn, optimizer,
                                                   patience=100, max_epochs=1000,
                                                   save_path=f"best_GraphSAGE.pt")

print(f"Test Accuracy: {acc:.3f}")
print(f"Test AUC:      {auc:.3f}")

fig, axes = plt.subplots(1, 3, figsize=(24, 6))
# Plot Loss Curves
axes[0].plot(losses, label="Train")
axes[0].plot(val_losses, label="Val")
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Loss")
axes[0].set_title("GAT Loss Curves")
axes[0].legend()

# Plot ROC Curve
axes[1].plot(fpr, tpr, lw=2, label=f'AUC = {auc:.3f}')
axes[1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
axes[1].set_xlim([0.0, 1.0])
axes[1].set_ylim([0.0, 1.05])
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('GAT Receiver Operating Characteristic (ROC) Curve')
axes[1].legend(loc="lower right")
axes[1].grid(True)

# Plot Confusion Matrix
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', ax=axes[2])
axes[2].set_xlabel('Predicted')
axes[2].set_ylabel('True')
axes[2].set_title('GAT Confusion Matrix')

plt.tight_layout()
plt.show()

We have now trained the three models on our network data. The high performance indicates that there is indeed financial contagtion happening faciliated by borrowers in the same area or having the same provider (or a combination of both).